In [ ]:
import _dataset_tools as dt
import _my_tools as mt
import cv2 as cv
import numpy as np
import os

### Image cropping
We take all of the grayscale radar images from folder `datasets/dataset_full/` and we crop squares of size $96\times96$ from them with a stride of size $48$. The cropped squares are saved in `datasets/cropped_96_96/` in folder according to the square's coordinates.

In [ ]:
out_f = "datasets/cropped_96_96/"
in_f = "datasets/dataset_full/"

for f in range(55):
    os.mkdir(out_f+str(f).zfill(2))

In [ ]:
dt.cropFolder(in_f,out_f,96,48)

### Cleaning and Triplets finding


In [ ]:
# this harvests all of the images into NP array

X = []
y = []

folder = "datasets_other/datasets_cropped/"
folders = sorted(os.listdir(folder))

for f in folders:
    images = sorted(os.listdir(folder+f+"/"))
    for i in range(0,len(images),3):
        img = [cv.imread(folder+f+"/"+images[j],cv.IMREAD_GRAYSCALE) for j in range(i,i+3)]
        x_tmp = np.stack([img[0],img[2]],axis=0)
        X.append(x_tmp)
        y.append(img[1])
        
X = np.array(X)
y = np.array(y)

In [ ]:
img = cv.imread("tmp/full.png",cv.IMREAD_GRAYSCALE)
img2 = cv.imread("tmp/gray.png",cv.IMREAD_GRAYSCALE)

X = np.array((64,64,2))
y = np.array((64,64,1))

np.append(X,[[img,img2]])
# X.append([img,img2])
X

In [ ]:
x = np.stack([img,img2],axis=0)
x2 = np.stack([img,img2],axis=0)
np.stack([x,x2,x],axis=0).shape

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtrash, ytrain, ytrash = train_test_split(X, y, test_size=0.35, random_state=123)
Xtrain, Xtest, ytrain, ytest = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=123)

In [ ]:
np.save("datasets/interpolation/X_train",Xtrain)
np.save("datasets/interpolation/y_train",ytrain)
np.save("datasets/interpolation/X_test",Xtest)
np.save("datasets/interpolation/y_test",ytest)

#### Showing images

In [ ]:
import os
import _my_tools as mt
import cv2 as cv

folder = "05/"

for i in sorted(os.listdir(folder)):
    if i[-4:] == ".png":
        img = cv.imread(folder+i,cv.IMREAD_GRAYSCALE)
        mt.showImg(i,img)